In [15]:
# JIT编译环境下的编程基础
'''
    JIT是一种运行计算机代码的方式，使程序在运行时而不是运行前完成编译
    JIT是一种动态编译的形式，允许自适应优化，开销小
'''

'\n    JIT是一种运行计算机代码的方式，使程序在运行时而不是运行前完成编译\n    JIT是一种动态编译的形式，允许自适应优化，开销小\n'

In [16]:
# 导库与环境设置
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt

bm.set_platform('cpu')  # 新数据存储在内存中，使用CPU计算（在有相关资源时，也可以设置为GPU或TPU）

In [17]:
# 1. JIT编译加速：利用brainpy.math.jit()包装目标函数或目标类，使Python代码转换成机器码

In [18]:
# 以高斯误差线性单元（GELU）函数为例
def gelu(x):
    sqrt = bm.sqrt(2 / bm.pi)
    cdf = 0.5 * (1.0 + bm.tanh(sqrt * (x + 0.044715 * (x ** 3))))
    y = x * cdf
    return y

In [19]:
# 不使用JIT编译，测试执行时间
x = bm.random.random(100000)        # 定义一个有100000个元素的随机输入数组
%timeit gelu(x)                     # 测试执行时间（注意%timeit只能在iPython环境，比如jupyter notebook下使用）

419 µs ± 39.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
# 使用JIT编译，测试执行时间
gelu_jit = bm.jit(gelu)             # 使用JIT
%timeit gelu_jit(x)                 # 测试执行时间

58.9 µs ± 415 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [21]:
# JIT也可以编译类和对象，规则如下：
'''
    1. 该类对象必须是brainpy.BrainPyObject的子类；
    2. 动态变量必须被定义为brainpy.math.Variable
'''

'\n    1. 该类对象必须是brainpy.BrainPyObject的子类；\n    2. 动态变量必须被定义为brainpy.math.Variable\n'

In [22]:
# 以逻辑回归为例：1. 权重w需要在训练中被修改，所以定义为brainpy.math.Variable；2. 其余变量均为静态变量，因此不会改变

In [23]:
# 类继承于brainpy.BrainPyObject
class LogisticRegression(bp.BrainPyObject):
    def __init__(self, dimension):
        super(LogisticRegression, self).__init__()

        # 参数定义
        self.dimension = dimension

        # 动态变量
        self.w = bm.Variable(2.0 * bm.ones(dimension) - 1.3)

    def __call__(self, X, Y):
        u = bm.dot(((1.0 / (1.0 + bm.exp(- Y * bm.dot(X, self.w))) - 1.0) * Y), X)
        self.w.value = self.w - u   # 动态更新参数

In [24]:
# 测试时间
import time

# 创建评价标准
def benchmark(model, points, labels, num_iter=30, name=''):
    t0 = time.time()
    for i in range(num_iter):
        model(points, labels)

    print(f'{name} Used Time {time.time()-t0}s')

# 定义数据集和标签
num_dim, num_points = 10, 20000000
points = bm.random.random((num_points, num_dim))
labels = bm.random.random(num_points)

In [25]:
# 测试不使用JIT需要的时间
lr1 = LogisticRegression(num_dim)
benchmark(lr1, points, labels, name='Logistic Regression Without JIT')

Logistic Regression Without JIT Used Time 6.614258766174316s


In [26]:
# 测试使用JIT需要的时间
lr2 = LogisticRegression(num_dim)
lr2 = bm.jit(lr2)
benchmark(lr2, points, labels, name='Logistic Regression With JIT')

Logistic Regression With JIT Used Time 4.083987236022949s


In [27]:
# brainpy.Runner是模拟、训练、积分等运行器的基类
# 在初始化时，运行器会收到名为jit的参数，默认True，表明Runner会自动编译目标工程

In [28]:
# 以HH模型为例
model = bp.neurons.HH(1000) # 1000个神经元的模型

# jit默认设置为True
runner1 = bp.DSRunner(target=model, inputs=('input', 10.))
runner1(duration=1000, eval_time=True)   # 模拟1000ms

  0%|          | 0/10000 [00:00<?, ?it/s]

(0.5234489440917969,
 Array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]], dtype=bool))

In [29]:
# 调试程序时，需要关闭JIT加速，方法是修改jit参数为False
runner2 = bp.DSRunner(target=model, inputs=('input', 10.), jit=False)
runner2(duration=1000, eval_time=True)   # 模拟1000ms

  0%|          | 0/10000 [00:00<?, ?it/s]

(710.564234495163,
 Array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]], dtype=bool))